In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data, color
from skimage.filters import gaussian
from scipy.ndimage import map_coordinates

def active_contour(image, snake, alpha=0.1, beta=0.1, gamma=0.1, 
                   iterations=250, w_line=0, w_edge=1.0, 
                   sigma=1.0, convergence=0.1):
    """
    Active contour model (Snake) for image segmentation.
    
    Parameters:
    - image: Input 2D image (grayscale).
    - snake: Initial snake coordinates (N x 2 array).
    - alpha: Snake length shape parameter (continuity).
    - beta: Snake smoothness shape parameter (curvature).
    - gamma: Step size.
    - iterations: Number of iterations to optimize the snake.
    - w_line: Line energy weight.
    - w_edge: Edge energy weight.
    - sigma: Gaussian smoothing parameter for edge energy.
    - convergence: Threshold to stop iterations if the snake converges.
    
    Returns:
    - snake: Optimized snake coordinates.
    """
    snake = np.array(snake, dtype=np.float32)
    n_points = len(snake)
    
    # Compute the image forces (edge-based energy)
    smoothed_image = gaussian(image, sigma)
    gx, gy = np.gradient(smoothed_image)
    edge_energy = np.hypot(gx, gy)
    
    # Matrix of coefficients for internal forces (alpha, beta)
    A = np.roll(np.eye(n_points), -1, axis=0) + np.roll(np.eye(n_points), 1, axis=0) - 2 * np.eye(n_points)
    B = np.roll(np.eye(n_points), -2, axis=0) + np.roll(np.eye(n_points), 2, axis=0) - 4 * np.roll(np.eye(n_points), -1, axis=0) - 4 * np.roll(np.eye(n_points), 1, axis=0) + 6 * np.eye(n_points)
    P = alpha * A - beta * B
    
    # Regularization matrix
    inv = np.linalg.inv(np.eye(n_points) - gamma * P)
    
    for _ in range(iterations):
        # Interpolate the image energy at snake points
        int_x = map_coordinates(edge_energy, [snake[:, 1], snake[:, 0]], order=1, mode='reflect')
        int_y = map_coordinates(edge_energy, [snake[:, 1], snake[:, 0]], order=1, mode='reflect')
        
        # External forces
        fx, fy = np.gradient(edge_energy)
        external_force = np.stack([map_coordinates(fx, [snake[:, 1], snake[:, 0]], order=1, mode='reflect'),
                                   map_coordinates(fy, [snake[:, 1], snake[:, 0]], order=1, mode='reflect')], axis=1)
        
        # Update the snake position
        snake += gamma * external_force
        snake = np.dot(inv, snake)
        
        # Convergence check
        if np.mean(np.sqrt(np.sum((snake - np.roll(snake, 1, axis=0))**2, axis=1))) < convergence:
            break
    
    return snake


Steps of active contour: 
- Initialize a shape (contour)
- Calculate the energy function:
    - internal energy calculation
    - external energy calculation
- Minimize the energy 
- calculate the new contour
- stop when convergence

In [ ]:
def initialize_contour(center, radius, points = 100):
    s = np.linespace(0, 2 * np.pi, points)
    x = center[0] + radius*np.cos(s)
    y = center[1] + radius*np.sin(s)
    
    return np.array([x,y]).T

def calculate_energy():
    'total energy = const* internal energy + const * external energy'
    pass

def calculate_internal_energy(n_points, alpha=0.1, beta=0.1, gamma=0.1):
    A = np.zeros((n_points, n_points))
    
    for i in range(n_points):
        A[i,i] = 2*alpha + 6*beta
        A[i, (i-1)%n_points] = -alpha - 4 * beta 
        A[i, (i+1)%n_points] = -alpha - 4 * beta
        A[i, (i-2)%n_points] = beta
        A[i, (i+2)%n_points] = beta
        
    Ainv = np.linalg.inv(A + gamma*np.eye(n_points))
    
    return Ainv

def calculate_external_energy(image, sigma =0.1):
    
    smoothed_image = gaussian_filter(image, sigma)
    gy , gx = np.gradient(smoothed_image)
    edge_energy =  np.sqrt(gx**2 + gy**2)
    
    return edge_energy, gy, gx

def optimize_contour(image, snake, inv_mat, gx, gy,
                      iterations, convergence, w_edge, gamma=0.1):
    for _ in range(iterations):
        int_x = np.clip(snake[:, 0].astype(int), 0, image.shape[1] -1)
        int_y = np.clip(snake[:, 1].astype(int), 0, image.shape[0] -1)
        
        fx = gx[int_y, int_x]
        fy = gy[int_y, int_x]
        
        force =np.stack([fx, fy], axis=1) * w_edge
        
        new_snake = np.dot(inv_mat, snake + gamma* force)
        
        if np.mean(np.sqrt(np.sum((new_snake- snake)**2, axis =1))) < convergence:
            break
        snake = new_snake
        
    return snake


def active_contour(image, center, radius, alpha=0.1, beta = 0.1, gamma=0.1, w_edge=0.1,
                   sigma =1, iterations = 250, convergence =0.1, points =100):
    
    snake = initialize_contour(center, radius, points)
    
    inv_matrix = calculate_internal_energy(len(snake), alpha, beta, gamma)
    
    edge_energy, gy, gx = calculate_external_energy(image, sigma)
    
    snake = optimize_contour(image, snake, inv_matrix, gx, gy, iterations,
                             convergence, w_edge, gamma)
    
    return snake 
    
    
